In [ ]:
#@title # ⚙️ 1.Tuỳ chọn phiên bản
#@markdown `🟢 Cập nhật 3.3, hỗ trợ đầy đủ cho Flux Tools ở chế độ Base - full` </br>
#@markdown </br>[![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/9rOWXzYtdGM) [![](https://img.shields.io/badge/Hướng%20dẫn%20-Sử%20dụng%20WebUI-5858e6)](https://stablediffusion.vn/huong-dan-chung/) [![](https://img.shields.io/badge/Tra%20cứu-Tính%20năng%20WebUI-green)](https://stablediffusion.vn/webui-catalog/) [![](https://img.shields.io/badge/Hướng%20dẫn%20-SD%20Nhập%20Môn-5858e6)](https://stablediffusion.vn/nhap-mon-stable-diffusion/)
from google.colab import drive
drive.mount('/content/drive')

#Config

SDVNFolder = '/content/SDVN-WebUI'
if 'Version' not in globals():
  !git clone https://github.com/sunflower2332/SDVN-WebUI {SDVNFolder}
  %env TF_CPP_MIN_LOG_LEVEL=1
  %env LD_PRELOAD={SDVNFolder}/libtcmalloc_minimal.so.4
  !sudo apt-get install aria2
  !wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !curl -sL https://tunnelto.dev/install.sh | sh
  # !npm install -g localtunnel
  test = 'test'

#Option
FastMode = True #@param {type:"boolean"}
UI_Version = "ComfyUI"
OptionMode = "☕️ base" # @param ["☕️ base","🍺 full","📸 +photo","🎥 +video","👨🏻‍💻 +dev"] {"allow-input":true}
Controlnet = "base" # @param ["none","base","+SD15","+SDXL","+Flux","User_list"] {"allow-input":true}
Fluxmodel = "Flux_dev_v1-fp8"  #@param ["", "Flux_dev_v1", "Flux_dev_v1_fill-fp8", "Flux_dev_v1-fp8", "Flux_dev_v1-nf4"] {allow-input: true}
#Install
%run /content/SDVN-WebUI/Script.ipynb

In [ ]:
#@title # ⌛️ 2.RUN
DriveLib = True
RunModels = True #@param {type:"boolean"}
CommandLine = "" #@param {type:"string"}
install_custom()
import time
import threading
from datetime import datetime, timedelta
import os, re

# Define a function to run the upload script in a separate thread
def run_upload_script():
    import time
    # Give ComfyUI a moment to start up
    time.sleep(10)
    print("\n\033[1;32mStarting Upload_image.ipynb for model management...\033[0m")
    
    # Get today's date in GMT+7 timezone
    now = datetime.utcnow() + timedelta(hours=7)  # GMT+7
    today = now.strftime("%Y-%m-%d")
    print(f"\033[1;32mImages will be saved to folder: {today} (GMT+7 timezone)\033[0m")
    
    # Create a dynamic version of Upload_image.ipynb with current date
    # First, dynamically create the current date folder
    todays_folder = f"/content/drive/MyDrive/SD-Data/Export/ComfyUI/{today}/"
    os.makedirs(todays_folder, exist_ok=True)
    
    # Create a dynamic modified version of Upload_image.ipynb with today's date
    src_path = "/content/SDVN-WebUI/Upload_image.ipynb"
    dst_path = "/tmp/dynamic_upload_image.ipynb"
    
    # Read the original file
    with open(src_path, "r", encoding="utf-8") as f:
        content = f.read()
    
    # Replace the hardcoded date with today's date
    content = re.sub(
        r'DEFAULT_UPLOAD_FOLDER = \'/content/drive/MyDrive/SD-Data/Export/ComfyUI/[^/]*\'',
        f'DEFAULT_UPLOAD_FOLDER = \'/content/drive/MyDrive/SD-Data/Export/ComfyUI/{today}/\'',
        content
    )
    content = re.sub(
        r'\.replace\(\'/content/drive/MyDrive/SD-Data/Export/ComfyUI/[^/]*/\'',
        f'.replace(\'/content/drive/MyDrive/SD-Data/Export/ComfyUI/{today}/\'',
        content
    )
    
    # Modify the CSS to show thumbnails in a 4-column grid
    # 1. Find the beginning of the style section
    style_start = content.find("<style>")
    style_end = content.find("</style>", style_start)
    
    if style_start != -1 and style_end != -1:
        # Extract the style section
        style_section = content[style_start:style_end]
        
        # Modify the file-list CSS for grid layout instead of scroll
        new_style_section = style_section.replace(
            ".file-list {\n            background-color: #f7f7f7;\n            padding: 15px;\n            border-radius: 5px;\n            margin-top: 20px;\n            max-height: 400px;\n            overflow-y: auto;\n        }",
            ".file-list {\n            background-color: #f7f7f7;\n            padding: 15px;\n            border-radius: 5px;\n            margin-top: 20px;\n            display: grid;\n            grid-template-columns: repeat(4, 1fr);\n            grid-gap: 15px;\n        }"
        )
        
        # Update file-item to display as cards in the grid
        new_style_section = new_style_section.replace(
            ".file-item {\n            display: flex;\n            justify-content: space-between;\n            align-items: center;\n            padding: 8px 0;\n            border-bottom: 1px solid #eee;\n        }",
            ".file-item {\n            display: flex;\n            flex-direction: column;\n            background-color: white;\n            border-radius: 8px;\n            box-shadow: 0 2px 4px rgba(0,0,0,0.1);\n            padding: 10px;\n            transition: transform 0.2s;\n        }\n        .file-item:hover {\n            transform: translateY(-5px);\n            box-shadow: 0 5px 15px rgba(0,0,0,0.1);\n        }"
        )
        
        # Update thumbnail size
        new_style_section = new_style_section.replace(
            ".file-thumbnail {\n            width: 450px;\n            height: 450px;\n            object-fit: cover;\n            margin-right: 30px;\n            border-radius: 10px;\n        }",
            ".file-thumbnail {\n            width: 100%;\n            height: 180px;\n            object-fit: cover;\n            border-radius: 6px;\n            margin-bottom: 10px;\n        }"
        )
        
        # Update file info display
        new_style_section = new_style_section.replace(
            ".file-info {\n            flex-grow: 1;\n            margin-right: 15px;\n            word-break: break-all;\n        }",
            ".file-info {\n            font-size: 12px;\n            margin-bottom: 10px;\n            word-break: break-all;\n            max-height: 50px;\n            overflow: hidden;\n            text-overflow: ellipsis;\n        }"
        )
        
        # Update file actions for buttons
        new_style_section = new_style_section.replace(
            ".file-actions {\n            display: flex;\n            justify-content: space-between;\n            gap: 10px;\n            margin-top: 5px;\n        }",
            ".file-actions {\n            display: flex;\n            justify-content: space-between;\n            gap: 5px;\n            margin-top: auto;\n        }"
        )
        
        # Make buttons smaller
        new_style_section = new_style_section.replace(
            "button, .btn {\n            background-color: #4285f4;\n            color: white;\n            border: none;\n            padding: 8px 16px;\n            border-radius: 4px;\n            cursor: pointer;\n            font-size: 14px;\n            text-decoration: none;\n            display: inline-block;\n        }",
            "button, .btn {\n            background-color: #4285f4;\n            color: white;\n            border: none;\n            padding: 5px 10px;\n            border-radius: 4px;\n            cursor: pointer;\n            font-size: 12px;\n            text-decoration: none;\n            display: inline-block;\n            flex: 1;\n            text-align: center;\n        }"
        )
        
        # Replace the style section in the content
        content = content[:style_start] + "<style>\n" + new_style_section[len("<style>\n"):] + content[style_end:]
    
    # Write the modified file
    with open(dst_path, "w", encoding="utf-8") as f:
        f.write(content)
    
    print(f"Created dynamic version of Upload_image.ipynb with current date: {today} and 4-column thumbnail grid")
    
    # Run the modified version
    %run /tmp/dynamic_upload_image.ipynb

# Start the upload script in a separate thread if enabled
if RunModels:
    thread = threading.Thread(target=run_upload_script)
    thread.daemon = True
    thread.start()
    print("\033[1;33mModel uploader will start shortly in the background...\033[0m")

# Run ComfyUI
run_ver(Version, CommandLine)
